|Name|Mnr|
|-|-|
|Joel Bück|4860895|
|Lukas Runge|7590014|
|Lukas Stamm|8402366|
|Martin Schauer|7961802|

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.metrics import make_scorer, mean_absolute_error
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from scipy.stats import chi2_contingency, pearsonr
from xgboost import XGBRegressor

# 1. Preprocessing & Exploration

In [ ]:
og_df = pd.read_csv('../AmesHousingForecast/ames.csv')
og_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 81 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   MS_SubClass         2930 non-null   object 
 1   MS_Zoning           2930 non-null   object 
 2   Lot_Frontage        2930 non-null   int64  
 3   Lot_Area            2930 non-null   int64  
 4   Street              2930 non-null   object 
 5   Alley               2930 non-null   object 
 6   Lot_Shape           2930 non-null   object 
 7   Land_Contour        2930 non-null   object 
 8   Utilities           2930 non-null   object 
 9   Lot_Config          2930 non-null   object 
 10  Land_Slope          2930 non-null   object 
 11  Neighborhood        2930 non-null   object 
 12  Condition_1         2930 non-null   object 
 13  Condition_2         2930 non-null   object 
 14  Bldg_Type           2930 non-null   object 
 15  House_Style         2930 non-null   object 
 16  Overal

In [ ]:
og_df.head()

,MS_SubClass,MS_Zoning,Lot_Frontage,Lot_Area,Street,Alley,Lot_Shape,Land_Contour,Utilities,Lot_Config,...,Fence,Misc_Feature,Misc_Val,Mo_Sold,Year_Sold,Sale_Type,Sale_Condition,Sale_Price,Longitude,Latitude
0,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,141,31770,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Corner,...,No_Fence,NaN,0,5,2010,WD,Normal,215000,-93.619754,42.054035
1,One_Story_1946_and_Newer_All_Styles,Residential_High_Density,80,11622,Pave,No_Alley_Access,Regular,Lvl,AllPub,Inside,...,Minimum_Privacy,NaN,0,6,2010,WD,Normal,105000,-93.619756,42.053014
2,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,81,14267,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Corner,...,No_Fence,Gar2,12500,6,2010,WD,Normal,172000,-93.619387,42.052659
3,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,93,11160,Pave,No_Alley_Access,Regular,Lvl,AllPub,Corner,...,No_Fence,NaN,0,4,2010,WD,Normal,244000,-93.617320,42.051245
4,Two_Story_1946_and_Newer,Residential_Low_Density,74,13830,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Inside,...,Minimum_Privacy,NaN,0,3,2010,WD,Normal,189900,-93.638933,42.060899


In [ ]:
na_count = og_df.isnull().sum()

na_count = na_count[na_count > 0]

print(na_count)

Mas_Vnr_Type    1775
Misc_Feature    2824
dtype: int64


In [ ]:

# Splitting the data into training and testing sets without stratification
features_train, features_test, target_train, target_test = train_test_split(og_df.drop(columns=["Sale_Price"]), og_df["Sale_Price"], test_size=0.3333, random_state=42)

#drop column misc_feature beacuse misc_value contains the exact same information
features_train = features_train.drop(columns=["Misc_Feature"])
features_test = features_test.drop(columns=["Misc_Feature"])

features_train[['Mas_Vnr_Type', 'Mas_Vnr_Area']] = features_train[['Mas_Vnr_Type', 'Mas_Vnr_Area']].fillna('NA')
features_test[['Mas_Vnr_Type', 'Mas_Vnr_Area']] = features_test[['Mas_Vnr_Type', 'Mas_Vnr_Area']].fillna('NA')



In [ ]:
# seperate to numerical and categorical columns
categorical_columns = features_train.select_dtypes(include = 'object').columns
numerical_columns = features_train.select_dtypes(include = 'number').columns

print(f'Categorical Count: {len(categorical_columns)} \n Numeric Count: {len(numerical_columns)}')

Categorical Count: 45 
 Numeric Count: 34


In [ ]:
insignificant_columns = []


In [ ]:
# Function to perform the Chi-Square test for categorical variables
def chi2_test(columns, features, target):
    results = {}
    for var in columns:
        contingency_table = pd.crosstab(features[var], target)
        chi2, p, dof, expected  = chi2_contingency(contingency_table)
        results[var] = {'Chi2': chi2, 'p-value': p}
        
    return results

# Store the results of the Chi-Square test for each categorical variable
results_ws = chi2_test(categorical_columns, features_train[categorical_columns], target_train)

for var, result in results_ws.items():
    print(f"Chi-Square Test for {var} and Sale_Price:\n  Chi2 = {result['Chi2']}, p-value = {result['p-value']} \n")
    
    # If the p-value is greater than 0.05, add the variable to the list of columns to remove
    if result['p-value'] > 0.05:
        insignificant_columns.append(var)


insignificant_columns


Chi-Square Test for MS_SubClass and Sale_Price:
  Chi2 = 10563.990692907704, p-value = 0.9999738059421928 

Chi-Square Test for MS_Zoning and Sale_Price:
  Chi2 = 7709.2558239148475, p-value = 2.391936563041048e-142 

Chi-Square Test for Street and Sale_Price:
  Chi2 = 1526.413815867612, p-value = 2.5626771763513798e-48 

Chi-Square Test for Alley and Sale_Price:
  Chi2 = 1490.968397544293, p-value = 0.9681146445359857 

Chi-Square Test for Lot_Shape and Sale_Price:
  Chi2 = 2851.8699667072547, p-value = 1.6038098441232264e-10 

Chi-Square Test for Land_Contour and Sale_Price:
  Chi2 = 2582.6200253261413, p-value = 0.0033658900421201887 

Chi-Square Test for Utilities and Sale_Price:
  Chi2 = 1369.9853846153846, p-value = 0.999983969727973 

Chi-Square Test for Lot_Config and Sale_Price:
  Chi2 = 3294.62361361067, p-value = 0.09191992032821766 

Chi-Square Test for Land_Slope and Sale_Price:
  Chi2 = 1775.2697997875187, p-value = 0.0009419814485799178 

Chi-Square Test for Neighborhood

['MS_SubClass',
 'Alley',
 'Utilities',
 'Lot_Config',
 'Condition_1',
 'Bldg_Type',
 'House_Style',
 'Roof_Style',
 'Roof_Matl',
 'Exterior_2nd',
 'Mas_Vnr_Type',
 'Bsmt_Cond',
 'BsmtFin_Type_1',
 'BsmtFin_Type_2',
 'Garage_Cond',
 'Paved_Drive',
 'Pool_QC',
 'Fence']

In [ ]:
# Calculate the Pearson correlation and p-values for each numeric column
correlation_pvalues = {}
for col in numerical_columns:
    corr, p_value = pearsonr(features_train[col], target_train)
    correlation_pvalues[col] = (corr, p_value)

# Display the correlation values and p-values
for col, (corr, p_value) in correlation_pvalues.items():
    print(f"{col}: correlation = {corr}, p-value = {p_value}")

# Add columns with p-value over 0.05 to the list of insignificant columns
for col, (corr, p_value) in correlation_pvalues.items():
    if p_value > 0.05:
        insignificant_columns.append(col)


insignificant_columns



Lot_Frontage: correlation = 0.18632989078798848, p-value = 1.0240298738132265e-16
Lot_Area: correlation = 0.2603638891730972, p-value = 1.2564134838812965e-31
Year_Built: correlation = 0.5382432636436444, p-value = 4.011049385733298e-147
Year_Remod_Add: correlation = 0.5149467084584336, p-value = 1.0223848778823359e-132
Mas_Vnr_Area: correlation = 0.4974547697612972, p-value = 1.2978966892558e-122
BsmtFin_SF_1: correlation = -0.14971237350432723, p-value = 2.941399310067451e-11
BsmtFin_SF_2: correlation = 0.03836792273273174, p-value = 0.09005209688592127
Bsmt_Unf_SF: correlation = 0.17230489925961961, p-value = 1.758399684032972e-14
Total_Bsmt_SF: correlation = 0.6159422025790819, p-value = 2.3180027931617903e-204
First_Flr_SF: correlation = 0.6079511962524037, p-value = 9.885220386015982e-198
Second_Flr_SF: correlation = 0.2900801912435975, p-value = 3.573669321294009e-39
Low_Qual_Fin_SF: correlation = -0.013695013051843194, p-value = 0.5452711569672127
Gr_Liv_Area: correlation = 0.7

['MS_SubClass',
 'Alley',
 'Utilities',
 'Lot_Config',
 'Condition_1',
 'Bldg_Type',
 'House_Style',
 'Roof_Style',
 'Roof_Matl',
 'Exterior_2nd',
 'Mas_Vnr_Type',
 'Bsmt_Cond',
 'BsmtFin_Type_1',
 'BsmtFin_Type_2',
 'Garage_Cond',
 'Paved_Drive',
 'Pool_QC',
 'Fence',
 'BsmtFin_SF_2',
 'Low_Qual_Fin_SF',
 'Bsmt_Half_Bath',
 'Three_season_porch',
 'Misc_Val',
 'Mo_Sold']

In [ ]:
#select only relevant columns

features_test = features_test.drop(insignificant_columns, axis=1)
features_train = features_train.drop(insignificant_columns, axis=1)

categorical_columns = list(set(categorical_columns)-set(insignificant_columns))
numerical_columns = list(set(numerical_columns)-set(insignificant_columns))

In [ ]:
features_test

,MS_Zoning,Lot_Frontage,Lot_Area,Street,Lot_Shape,Land_Contour,Land_Slope,Neighborhood,Condition_2,Overall_Qual,...,Wood_Deck_SF,Open_Porch_SF,Enclosed_Porch,Screen_Porch,Pool_Area,Year_Sold,Sale_Type,Sale_Condition,Longitude,Latitude
1357,Residential_Medium_Density,0,5100,Pave,Regular,Lvl,Gtl,Old_Town,Norm,Very_Good,...,192,63,0,0,0,2008,WD,Normal,-93.621065,42.029038
2367,Residential_Medium_Density,21,1890,Pave,Regular,Lvl,Gtl,Briardale,Norm,Above_Average,...,0,0,0,0,0,2006,WD,Normal,-93.627103,42.051798
2822,Residential_Low_Density,62,7162,Pave,Regular,Lvl,Gtl,Edwards,Norm,Good,...,168,57,0,0,0,2006,WD,Normal,-93.672379,42.018990
2126,Residential_Low_Density,60,8070,Pave,Regular,Lvl,Gtl,College_Creek,Norm,Below_Average,...,0,0,0,0,0,2007,WD,Normal,-93.692415,42.019028
1544,Residential_Medium_Density,50,7000,Pave,Regular,Lvl,Gtl,Iowa_DOT_and_Rail_Road,Norm,Above_Average,...,0,0,116,0,0,2008,WD,Normal,-93.628409,42.022607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016,Residential_Medium_Density,51,6120,Pave,Regular,Lvl,Gtl,Brookside,Norm,Average,...,25,0,0,0,0,2007,WD,Normal,-93.621504,42.032505
2681,Residential_Medium_Density,51,6120,Pave,Regular,Lvl,Gtl,Brookside,Norm,Average,...,0,0,55,0,0,2006,WD,Normal,-93.622415,42.032418
2129,Residential_Low_Density,43,12352,Pave,Moderately_Irregular,Lvl,Gtl,College_Creek,Norm,Good,...,0,74,0,0,0,2007,WD,Normal,-93.688920,42.017826
969,Residential_Low_Density,75,12518,Pave,Slightly_Irregular,Lvl,Gtl,Mitchell,Norm,Average,...,144,0,0,0,0,2009,WD,Normal,-93.604264,41.993540


In [ ]:
features_train

,MS_Zoning,Lot_Frontage,Lot_Area,Street,Lot_Shape,Land_Contour,Land_Slope,Neighborhood,Condition_2,Overall_Qual,...,Wood_Deck_SF,Open_Porch_SF,Enclosed_Porch,Screen_Porch,Pool_Area,Year_Sold,Sale_Type,Sale_Condition,Longitude,Latitude
1498,Residential_Low_Density,313,63887,Pave,Irregular,Bnk,Gtl,Edwards,Norm,Very_Excellent,...,214,292,0,0,480,2008,New,Partial,-93.674898,42.016804
1766,Residential_Low_Density,83,10790,Pave,Regular,Lvl,Gtl,Northridge,Norm,Good,...,120,38,0,0,0,2007,WD,Normal,-93.649901,42.053467
2192,Residential_Low_Density,60,8172,Pave,Regular,HLS,Gtl,Edwards,Norm,Below_Average,...,196,0,0,0,0,2007,COD,Family,-93.663455,42.018730
192,Residential_Low_Density,0,7793,Pave,Slightly_Irregular,Bnk,Gtl,Brookside,Norm,Good,...,0,0,60,0,0,2010,WD,Normal,-93.625825,42.030187
801,Residential_Low_Density,79,12327,Pave,Slightly_Irregular,Low,Mod,Sawyer_West,Norm,Very_Good,...,349,40,0,0,0,2009,WD,Normal,-93.685986,42.031845
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638,Residential_Low_Density,195,41600,Pave,Slightly_Irregular,Lvl,Gtl,Gilbert,Norm,Average,...,144,0,0,0,0,2007,WD,Normal,-93.622874,42.060096
1095,Residential_Low_Density,0,7750,Pave,Regular,Lvl,Gtl,Gilbert,Norm,Good,...,100,35,0,0,0,2008,WD,Normal,-93.643632,42.059507
1130,Floating_Village_Residential,64,8791,Pave,Slightly_Irregular,Lvl,Gtl,Somerset,Norm,Above_Average,...,216,56,0,0,0,2008,WD,Normal,-93.639662,42.050899
1294,Residential_Medium_Density,81,7308,Pave,Regular,Lvl,Gtl,Old_Town,Norm,Average,...,0,0,236,0,0,2008,WD,Normal,-93.620219,42.030482


In [ ]:
categorical_columns

['Land_Contour',
 'Central_Air',
 'Neighborhood',
 'Fireplace_Qu',
 'Heating',
 'Garage_Type',
 'Sale_Type',
 'Kitchen_Qual',
 'MS_Zoning',
 'Heating_QC',
 'Overall_Qual',
 'Exterior_1st',
 'Garage_Qual',
 'Electrical',
 'Condition_2',
 'Bsmt_Qual',
 'Garage_Finish',
 'Functional',
 'Overall_Cond',
 'Land_Slope',
 'Street',
 'Exter_Qual',
 'Foundation',
 'Bsmt_Exposure',
 'Lot_Shape',
 'Exter_Cond',
 'Sale_Condition']

In [ ]:
numerical_columns

['Second_Flr_SF',
 'Longitude',
 'Fireplaces',
 'Pool_Area',
 'Kitchen_AbvGr',
 'BsmtFin_SF_1',
 'Mas_Vnr_Area',
 'Year_Sold',
 'Open_Porch_SF',
 'Bedroom_AbvGr',
 'Gr_Liv_Area',
 'Garage_Cars',
 'Latitude',
 'Total_Bsmt_SF',
 'Screen_Porch',
 'Year_Remod_Add',
 'Lot_Area',
 'First_Flr_SF',
 'Bsmt_Full_Bath',
 'Wood_Deck_SF',
 'Full_Bath',
 'TotRms_AbvGrd',
 'Half_Bath',
 'Enclosed_Porch',
 'Year_Built',
 'Garage_Area',
 'Bsmt_Unf_SF',
 'Lot_Frontage']

# 2. Transformation

In [ ]:
label_encoder = LabelEncoder()

for col in categorical_columns:
    features_train[col] = label_encoder.fit_transform(features_train[col])
    features_test[col] = label_encoder.fit_transform(features_test[col])


print(features_train[categorical_columns].head())
print(features_test[categorical_columns].head())

      Land_Contour  Central_Air  Neighborhood  Fireplace_Qu  Heating  \
1498             0            1             7             2        1   
1766             3            1            17             5        1   
2192             1            1             7             3        1   
192              0            0             3             2        1   
801              2            1            22             2        1   

      Garage_Type  Sale_Type  Kitchen_Qual  MS_Zoning  Heating_QC  ...  \
1498            0          6             0          5           0  ...   
1766            2          8             2          5           0  ...   
2192            4          0             3          5           4  ...   
192             4          8             2          5           4  ...   
801             2          8             2          5           0  ...   

      Functional  Overall_Cond  Land_Slope  Street  Exter_Qual  Foundation  \
1498           7             1           0  

In [ ]:
models = {
    'XGBoost': (XGBRegressor(
        subsample=0.6,
        n_estimators=400,
        min_child_weight=5,
        max_depth=8,
        learning_rate=7.906043210907701e-05,
        gamma=0.3,
        colsample_bytree=0.7,
        random_state=42,
        objective="reg:squarederror"
    ), {
        'regressor__learning_rate': np.logspace(-5, -1, 50),  # Explore a wide range of learning rates
        'regressor__max_depth': [3, 4, 5, 6, 7, 8, 9, 10],   # Tree depth
        'regressor__min_child_weight': [1, 3, 5, 7],          # Minimum child weight
        'regressor__gamma': [0, 0.1, 0.2, 0.3, 0.4],          # Minimum loss reduction
        'regressor__subsample': [0.6, 0.7, 0.8, 0.9, 1.0],    # Sampling ratio
        'regressor__colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],  # Feature sampling ratio
        'regressor__n_estimators': [100, 200, 300, 400, 500]  # Number of estimators
    }),
    'RandomForest': (RandomForestRegressor(
        max_features='sqrt',
        n_estimators=100,
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        bootstrap=True,
        random_state=42
    ), {
        'regressor__n_estimators': [100, 200, 300, 400, 500],
        'regressor__max_depth': [None, 10, 20, 30, 40, 50],
        'regressor__min_samples_split': [2, 5, 10],
        'regressor__min_samples_leaf': [1, 2, 4],
        'regressor__max_features': [1, 5, 10, 50, 'sqrt', 'log2'],
        'regressor__bootstrap': [True, False]
    }),
}


In [ ]:
class OutlierRemover(BaseEstimator, TransformerMixin):
    def __init__(self, method='iqr', multiplier=1.5):
        self.method = method
        self.multiplier = multiplier
        self.lower_bound = None
        self.upper_bound = None
    
    def fit(self, X, y=None):
        if self.method == 'iqr':
            Q1 = X.quantile(0.25)
            Q3 = X.quantile(0.75)
            IQR = Q3 - Q1
            self.lower_bound = Q1 - self.multiplier * IQR
            self.upper_bound = Q3 + self.multiplier * IQR
        return self
    
    def transform(self, X, y=None):
        if self.method == 'iqr':
            X_clipped = X.clip(lower=self.lower_bound, upper=self.upper_bound, axis=1)
            return X_clipped
        return X

In [ ]:
def cross_validate_best_model(model_name, best_model, features, target, cv=10, scoring=mse_scorer):
    best_params = best_model.best_params_
    regressor = best_model.best_estimator_['regressor']

    pipeline = Pipeline(steps=[
        ('processor', preprocessor),
        ('regressor', model)
    ]) 

    cv_scores = cross_val_score(pipeline, features, target, cv=cv, scoring=scoring, n_jobs=-1)

    print(f'{model_name} Cross-Validation MSE: {np.mean(cv_scores):.2f} ± {np.std(cv_scores):.2f}')

    return cv_scores

NameError: name 'mse_scorer' is not defined

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('remove_outliers', OutlierRemover(method='iqr', multiplier=1.5), numerical_columns), 
    ],
    remainder='passthrough'
)

mse_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

def fit_model(model_name, model, param_grid, features_train, target_train, scoring, cv=10, n_iter=100, random_state=42, n_jobs=-1):
    
    pipeline = Pipeline(steps=[
        ('processor', preprocessor),
        ('regressor', model)
    ])
    
    randomized_search = RandomizedSearchCV(estimator=pipeline, param_distributions=param_grid, scoring=scoring, cv=cv, n_iter=n_iter, random_state=random_state, n_jobs=n_jobs)
    randomized_search.fit(features_train, target_train)
    
    best_score = -randomized_search.best_score_
    best_params = randomized_search.best_params_
    
    print(f'{model_name}:\nMSE : {best_score:.2f}%')
    print('Best Parameters :', best_params)
    print('\n')
    
    return randomized_search, best_params

best_models = {}
for model_name, (model, param_grid) in models.items():
    best_model, best_params = fit_model(model_name, model, param_grid, features_train, target_train, scoring=mse_scorer)
    best_models[model_name] = (best_model, best_params)

XGBoost:
MSE : 14424.55%
Best Parameters : {'regressor__subsample': 0.6, 'regressor__n_estimators': 400, 'regressor__min_child_weight': 3, 'regressor__max_depth': 5, 'regressor__learning_rate': 0.05689866029018293, 'regressor__gamma': 0.2, 'regressor__colsample_bytree': 1.0}


RandomForest:
MSE : 15728.38%
Best Parameters : {'regressor__n_estimators': 400, 'regressor__min_samples_split': 2, 'regressor__min_samples_leaf': 1, 'regressor__max_features': 10, 'regressor__max_depth': None, 'regressor__bootstrap': False}




In [ ]:
def cross_validate_best_model(model_name, best_model, features, target, cv=10, scoring=mse_scorer):
    best_params = best_model.best_params_
    regressor = best_model.best_estimator_['regressor']

    pipeline = Pipeline(steps=[
        ('processor', preprocessor),
        ('regressor', model)
    ]) 

    cv_scores = cross_val_score(pipeline, features, target, cv=cv, scoring=scoring, n_jobs=-1)

    print(f'{model_name} Cross-Validation MSE: {np.mean(-cv_scores):.2f} ± {np.std(-cv_scores):.2f}')

    return cv_scores

cv_scores_dict = {}
for model_name, (best_model, _) in best_models.items():
    cv_scores = cross_validate_best_model(model_name, best_model, features_train, target_train) 
    cv_scores_dict[model_name] = cv_scores

XGBoost Cross-Validation MSE: 16658.06 ± 1873.88
RandomForest Cross-Validation MSE: 16658.06 ± 1873.88


In [ ]:
pred = []

for model_name, (best_model, _) in best_models.items():
    prediction = best_model.predict(features_test)
    pred.append(prediction)
    
test_data = pd.DataFrame()
test_data['Sale_Price'] = target_test   
test_data['XGBoost'] = pred[0]
test_data['Random Forest'] = pred[1]

In [ ]:
test_data

,Sale_Price,XGBoost,Random Forest
1357,161000,167524.234375,159367.5525
2367,116000,105252.312500,104679.4450
2822,196500,192585.187500,196340.6750
2126,123600,128965.250000,130974.4150
1544,126000,118040.367188,115788.6875
...,...,...,...
2016,132500,133025.406250,120296.4700
2681,110000,135394.921875,120520.7150
2129,217000,213034.125000,208569.1650
969,139500,139248.750000,138758.6600
